1. Merge LaLiga Season CSV Files (2015-2025)
2. Keep only the columns: Date, HomeTeam, AwayTeam, FTHG, FTAG
3. Merges them into a single dataframe
4. Saves the result as `2015-2025.csv`

In [22]:
import pandas as pd
from pathlib import Path

# Setup paths
BASE_DIR = Path('..')
DATA_DIR = BASE_DIR / 'data'
OUTPUT_FILE = Path('.') / '2015-2025.csv'
data_path = "data_engineering/2015-2025.csv"

In [12]:
# Columns to keep
KEEP_COLS = ['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG']

print(f'Reading CSV files from: {DATA_DIR}')
print(f'Will save merged result to: {OUTPUT_FILE}')

Reading CSV files from: ..\data
Will save merged result to: 2015-2025.csv


In [13]:
# Read and merge all SP1 CSV files
dfs = []
for csv_file in sorted(DATA_DIR.glob('SP1*.csv')):
    print(f'Reading {csv_file.name}')
    df = pd.read_csv(csv_file)
    # Validate required columns exist
    missing = [col for col in KEEP_COLS if col not in df.columns]
    if missing:
        print(f'WARNING: {csv_file.name} missing columns: {missing}')
        continue
    # Keep only required columns
    df = df[KEEP_COLS].copy()
    dfs.append(df)

# Merge all dataframes
if dfs:
    merged = pd.concat(dfs, axis=0, ignore_index=True)
    print(f'\nMerged {len(dfs)} files, got {len(merged)} rows')
    
    # Save merged result
    merged.to_csv(OUTPUT_FILE, index=False)
    print(f'Saved to {OUTPUT_FILE}')
else:
    print('No valid CSV files found to merge!')

Reading SP1 2015.csv
Reading SP1 2016.csv
Reading SP1 2017.csv
Reading SP1 2018.csv
Reading SP1 2019.csv
Reading SP1 2020.csv
Reading SP1 2021.csv
Reading SP1 2022.csv
Reading SP1 2023.csv
Reading SP1 2024.csv
Reading SP1 2025.csv

Merged 11 files, got 3900 rows
Saved to 2015-2025.csv


Add "Season" column. Seasons usually begin in mid august and finish in late may

In [27]:
import pandas as pd

# Load CSV
data_path = "2015-2025.csv"
df = pd.read_csv(data_path)

# Ensure Date column is datetime
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True, errors='coerce')

def get_season(date):
    if pd.isnull(date):
        return None
    year = date.year
    month = date.month
    if month >= 8:
        return f"{str(year)[-2:]}/{str(year+1)[-2:]}"
    else:
        return f"{str(year-1)[-2:]}/{str(year)[-2:]}"

# Always update 'Season' column values
df['Season'] = df['Date'].apply(get_season)
# Move 'Season' right after 'Date', if not already
cols = list(df.columns)
season_idx = cols.index('Season')
date_idx = cols.index('Date')
# Only move if 'Season' is not just after 'Date'
if season_idx != date_idx + 1:
    cols.insert(date_idx + 1, cols.pop(season_idx))
    df = df[cols]

df.to_csv(data_path, index=False)
df.head()

C:\Users\mmeri\AppData\Local\Temp\ipykernel_21824\2783491840.py:8: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df['Date'] = pd.to_datetime(df['Date'], dayfirst=True, errors='coerce')


,Date,Season,HomeTeam,AwayTeam,FTHG,FTAG
0,2015-08-21,15/16,Malaga,Sevilla,0,0
1,2015-08-22,15/16,Ath Madrid,Las Palmas,1,0
2,2015-08-22,15/16,Espanol,Getafe,1,0
3,2015-08-22,15/16,La Coruna,Sociedad,0,0
4,2015-08-22,15/16,Vallecano,Valencia,0,0


In [28]:
import pandas as pd
import numpy as np

# Load the data
file_path = "2015-2025.csv"
df = pd.read_csv(file_path)

# Handling ELO rating column if it exists
elo_col_candidates = [col for col in df.columns if 'elo' in col.lower()]
elo_col = elo_col_candidates[0] if elo_col_candidates else None

# Helper to compute points from FTHG, FTAG for home/away
def calc_points(row, side):
    if row['FTHG'] > row['FTAG']:
        return 3 if side == 'home' else 0
    elif row['FTHG'] == row['FTAG']:
        return 1
    else:
        return 0 if side == 'home' else 3

records = []
for index, row in df.iterrows():
    # Home record
    home = {
        'team': row['HomeTeam'],
        'season': row['Season'],
        'is_home': 1,
        'match_result': 'W' if row['FTHG'] > row['FTAG'] else ('D' if row['FTHG'] == row['FTAG'] else 'L'),
        'goals_for': row['FTHG'],
        'goals_against': row['FTAG'],
        'pts': calc_points(row, 'home'),
        'elo': row[elo_col] if elo_col else np.nan
    }
    # Away record
    away = {
        'team': row['AwayTeam'],
        'season': row['Season'],
        'is_home': 0,
        'match_result': 'W' if row['FTAG'] > row['FTHG'] else ('D' if row['FTHG'] == row['FTAG'] else 'L'),
        'goals_for': row['FTAG'],
        'goals_against': row['FTHG'],
        'pts': calc_points(row, 'away'),
        'elo': row[elo_col] if elo_col else np.nan
    }
    records.extend([home, away])

long_df = pd.DataFrame(records)

# Aggregations per team/season
gb = long_df.groupby(['team', 'season'])
aggs = gb.agg(
    pts=('pts', 'sum'),
    wins=('match_result', lambda x: (x == 'W').sum()),
    draws=('match_result', lambda x: (x == 'D').sum()),
    losses=('match_result', lambda x: (x == 'L').sum()),
    win_rate=('match_result', lambda x: (x == 'W').mean()),
    draw_rate=('match_result', lambda x: (x == 'D').mean()),
    loss_rate=('match_result', lambda x: (x == 'L').mean()),
    goals_for=('goals_for', 'sum'),
    goals_against=('goals_against', 'sum'),
    goals_home=('goals_for', lambda x: x[long_df.loc[x.index, 'is_home'] == 1].sum()),
    goals_away=('goals_for', lambda x: x[long_df.loc[x.index, 'is_home'] == 0].sum()),
    home_pts=('pts', lambda x: x[long_df.loc[x.index, 'is_home'] == 1].sum()),
    away_pts=('pts', lambda x: x[long_df.loc[x.index, 'is_home'] == 0].sum()),
    pct_home_games=('is_home', 'mean'),
    elo_start_of_season=('elo', 'first'),
    elo_end_of_season=('elo', 'last'),
    elo_mean_of_season=('elo', 'mean'),
    total_games=('is_home', 'count')
).reset_index()

# Goal difference
aggs['goal_difference'] = aggs['goals_for'] - aggs['goals_against']
# Percentage of matches played at home (convert from mean of 1s/0s)
aggs['pct_home_games'] = aggs['pct_home_games']

# Champion flag per season
def mark_champions(subdf):
    # team(s) with max points, tiebreak via goal_diff
    max_pts = subdf['pts'].max()
    contenders = subdf[subdf['pts'] == max_pts]
    max_gd = contenders['goal_difference'].max()
    top = contenders[contenders['goal_difference'] == max_gd]
    chosen_idx = top.index[0]  # will pick the first in case of further ties
    subdf['is_champion'] = 0
    subdf.loc[chosen_idx, 'is_champion'] = 1
    return subdf

aggs = aggs.groupby('season', group_keys=False).apply(mark_champions)

# Sort and select final columns
final_cols = [
    'team', 'season', 'pts', 'wins', 'draws', 'losses',
    'win_rate', 'draw_rate', 'loss_rate', 'goals_for', 'goals_against',
    'goal_difference', 'goals_home', 'goals_away',
    'home_pts', 'away_pts', 'pct_home_games', 
    'elo_start_of_season', 'elo_end_of_season', 'elo_mean_of_season',
    'is_champion'
]
missing_cols = [col for col in final_cols if col not in aggs.columns]
for m in missing_cols:
    aggs[m] = np.nan
aggs = aggs[final_cols]

# Save as new CSV file overwriting the old one
aggs.to_csv(file_path, index=False)
aggs.head()


C:\Users\mmeri\AppData\Local\Temp\ipykernel_21824\2806638611.py:89: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  aggs = aggs.groupby('season', group_keys=False).apply(mark_champions)


,team,season,pts,wins,draws,losses,win_rate,draw_rate,loss_rate,goals_for,...,goal_difference,goals_home,goals_away,home_pts,away_pts,pct_home_games,elo_start_of_season,elo_end_of_season,elo_mean_of_season,is_champion
0,Alaves,16/17,55,14,13,11,0.368421,0.342105,0.289474,41,...,-2,19,22,29,26,0.5,NaN,NaN,NaN,0
1,Alaves,17/18,47,15,2,21,0.394737,0.052632,0.552632,40,...,-10,21,19,28,19,0.5,NaN,NaN,NaN,0
2,Alaves,18/19,50,13,11,14,0.342105,0.289474,0.368421,39,...,-11,19,20,29,21,0.5,NaN,NaN,NaN,0
3,Alaves,19/20,39,10,9,19,0.263158,0.236842,0.500000,34,...,-25,20,14,27,12,0.5,NaN,NaN,NaN,0
4,Alaves,20/21,38,9,11,18,0.236842,0.289474,0.473684,36,...,-21,21,15,24,14,0.5,NaN,NaN,NaN,0


In [1]:
# Remove the pct_home_games column as it is always 0.5
import pandas as pd

df = pd.read_csv("2015-2025.csv")
if 'pct_home_games' in df.columns:
    df = df.drop(columns=["pct_home_games"])
    df.to_csv("2015-2025.csv", index=False)
display(df)

,team,season,pts,wins,draws,losses,win_rate,draw_rate,loss_rate,goals_for,goals_against,goal_difference,goals_home,goals_away,home_pts,away_pts,is_champion,elo_start_of_season_elo,elo_end_of_season_elo,elo_mean_of_season_elo
0,Alaves,16/17,55,14,13,11,0.368421,0.342105,0.289474,41,43,-2,19,22,29,26,0,1300.000000,1347.240342,1309.374489
1,Alaves,17/18,47,15,2,21,0.394737,0.052632,0.552632,40,50,-10,21,19,28,19,0,1347.240342,1306.381991,1284.339356
2,Alaves,18/19,50,13,11,14,0.342105,0.289474,0.368421,39,50,-11,19,20,29,21,0,1306.381991,1299.643613,1335.265079
3,Alaves,19/20,39,10,9,19,0.263158,0.236842,0.500000,34,59,-25,20,14,27,12,0,1299.643613,1249.263197,1285.863602
4,Alaves,20/21,38,9,11,18,0.236842,0.289474,0.473684,36,57,-21,21,15,24,14,0,1249.263197,1264.336068,1246.566436
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,Villarreal,21/22,59,16,11,11,0.421053,0.289474,0.289474,63,37,26,40,23,36,23,0,1375.713259,1391.138858,1376.581102
216,Villarreal,22/23,64,19,7,12,0.500000,0.184211,0.315789,59,40,19,36,23,39,25,0,1391.138858,1415.522606,1395.199035
217,Villarreal,23/24,53,14,11,13,0.368421,0.289474,0.342105,65,65,0,36,29,26,27,0,1415.522606,1400.313795,1354.969900
218,Villarreal,24/25,70,20,10,8,0.526316,0.263158,0.210526,71,51,20,43,28,34,36,0,1400.313795,1472.200438,1419.756083
